In [1]:
import re
from functools import cache
from sympy import symbols, Eq, solve

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [3]:
# parse games
games = []
for i in range(0, len(lines), 4):
    game = {}
    button_a = (
        int(re.findall(r"X\+(\d+)", lines[i])[0]),
        int(re.findall(r"Y\+(\d+)", lines[i])[0]),
    )
    button_b = (
        int(re.findall(r"X\+(\d+)", lines[i + 1])[0]),
        int(re.findall(r"Y\+(\d+)", lines[i + 1])[0]),
    )
    target = (
        int(re.findall(r"X=(\d+)", lines[i + 2])[0]),
        int(re.findall(r"Y=(\d+)", lines[i + 2])[0]),
    )
    # game["a"] = button_a
    # game["b"] = button_b
    # game["target"] = target
    # print(game)
    games.append((button_a, button_b, target))
# games

In [4]:
# button a: 3 tokens
# button b: 1 token

In [5]:
# When having a hammer, everything looks like a nail I guess, so I didnt bother with maths first and did some recursive search with cache
@cache
def search_min_tokens(game, curr_tokens, curr_state, button_presses):
    # print(game, curr_tokens, curr_state, button_presses)
    if (
        curr_state[0] > game[2][0]
        or curr_state[1] > game[2][1]
        or button_presses[0] > 100
        or button_presses[1] > 100
    ):
        return None

    if curr_state[0] == game[2][0] and curr_state[1] == game[2][1]:
        return curr_tokens

    press_a = search_min_tokens(
        game,
        curr_tokens + 3,
        (curr_state[0] + game[0][0], curr_state[1] + game[0][1]),
        (button_presses[0] + 1, button_presses[1]),
    )

    press_b = search_min_tokens(
        game,
        curr_tokens + 1,
        (curr_state[0] + game[1][0], curr_state[1] + game[1][1]),
        (button_presses[0], button_presses[1] + 1),
    )
    # print(press_a, press_b)
    if press_a and press_b:
        return min(press_a, press_b)
    if press_a:
        return press_a
    if press_b:
        return press_b
    return None


search_min_tokens(games[0], 0, (0, 0), (0, 0))

In [ ]:
game_sum = 0
for game in games:
    g = search_min_tokens(game, 0, (0, 0), (0, 0))
    if g:
        game_sum += g
game_sum

In [7]:
# part 2

In [8]:
# Recursion explodes, so use good old maths and a solver
# target_x = button_a_x * a + button_b_x * b
# target_y = button_a_y * a + button_b_y * b

In [9]:
def solve_button_equation(input_data):
    (button_a_x, button_a_y), (button_b_x, button_b_y), (target_x, target_y) = (
        input_data
    )

    a, b = symbols("a b", integer=True)

    eq1 = Eq(button_a_x * a + button_b_x * b, target_x)
    eq2 = Eq(button_a_y * a + button_b_y * b, target_y)

    solutions = solve((eq1, eq2))

    return solutions

In [ ]:
# Try first part to be sure
game_sum = 0
a, b = symbols("a b", integer=True)
for game in games:
    s = solve_button_equation(game)
    if s:
        game_sum += s[a] * 3 + s[b]
game_sum

In [ ]:
# do second part
game_sum = 0
a, b = symbols("a b", integer=True)
for game in games:
    game = (
        game[0],
        game[1],
        (game[2][0] + 10000000000000, game[2][1] + 10000000000000),
    )
    s = solve_button_equation(game)
    if s:
        game_sum += s[a] * 3 + s[b]
game_sum